In [ ]:
%matplotlib qt
import os
from pathlib import Path
import matplotlib.pyplot as plt
from tqdm import tqdm
from mne.io import read_raw_brainvision, read_raw_fif

from scipy.signal import find_peaks
import numpy as np
from mne.channels import read_dig_captrak, make_standard_montage
from mne.preprocessing import ICA
from mne_icalabel import label_components
import customtkinter as ctk
from mne.coreg import Coregistration
from mne import (open_report, events_from_annotations, Epochs, write_trans,
                setup_source_space, make_bem_model, make_bem_solution, make_forward_solution, compute_covariance,
                extract_label_time_course, read_labels_from_annot)
from mne.minimum_norm import make_inverse_operator, apply_inverse
from mne.datasets import fetch_fsaverage
from scipy.signal import find_peaks
import warnings
from mne.preprocessing import ICA, EOGRegression, create_eog_epochs, create_ecg_epochs, compute_proj_ecg, compute_proj_eog, EOGRegression

In [ ]:
%%latex
##### for preprocessing()
## ask philip where is PO7, PO8, O1 and O2 and adjust it here
## put these cahnnels as misc and see what we can do with it
## kernel crash when running ctk
## ssp for ecg (pulse)
## ssp for vertical eye
## regression for horizontal eye
## .load_data()
## add teas
## GSR, X, Y, Y drop
## rename y-axis in pulse plot maybe *1e9 and time shows
## mne.preprocessing.annotate_break
## interpolate bads (mne.preprocessing.find_bad_channels_lof)



##### for run_analysis()
## in gpias dont do baselining
## double check with repo codes
## ask gpias triggers code
## documenatiaion
## 

In [ ]:
%matplotlib qt
from eeg_preprocessing import preprocessing
preprocessing(subject_id="zdfy",
                subjects_dir="/Users/payamsadeghishabestari/antinomics_clean_codes/subjects",
                paradigm="gpias",
                manual_data_scroll=True,
                run_ica=False,
                manual_ica_removal=False,
                eog_correct=True,
                pulse_correct=True,
                resp_correct=False,
                create_report=True,
                saving_dir=None,
                verbose="ERROR")

In [ ]:
%matplotlib qt
from eeg_rs_processing import run_rs_analysis

run_rs_analysis(
        subject_id="zdfy",
        subjects_dir="/Users/payamsadeghishabestari/antinomics_clean_codes/subjects",
        visit=1,
        event_ids=None,
        source_analysis=True,
        mri=False,
        subjects_fs_dir=None,
        manual_data_scroll=False,
        create_report=True,
        saving_dir=None,
        verbose="ERROR"
        )


In [4]:
%matplotlib qt
from eeg_erp_processing import run_erp_analysis

run_erp_analysis(
        subject_id="zdfy",
        subjects_dir="/Users/payamsadeghishabestari/antinomics_clean_codes/subjects",
        paradigm="gpias",
        events=None,
        source_analysis=True,
        mri=False,
        subjects_fs_dir=None,
        manual_data_scroll=False,
        create_report=True,
        saving_dir=None,
        verbose="ERROR"
        )


                                                  
                                               

 20%|█████████                                    



Loading preprocessed EEG data ...



ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (3,) + inhomogeneous part.

In [9]:
import mne
import numpy as np
import matplotlib.pyplot as plt

In [79]:
raw = mne.io.read_raw("/Users/payamsadeghishabestari/antinomics_clean_codes/subjects/zdfy/EEG/gpias/raw_prep.fif", preload=True)
events, event_ids = mne.events_from_annotations(raw)
if "New Segment/" in event_ids: 
    events = events[events[:, 2] != event_ids["New Segment/"]]
    event_ids.pop("New Segment/")

In [72]:
events

array([[  1228,      0,      2],
       [108385,      0,      4],
       [365875,      0,      5],
       [623901,      0,      3],
       [882334,      0,      6]])

In [51]:
raw.plot()

In [7]:
def _detect_gpias_events(
                        stim_ch,
                        times,
                        stim_key,
                        plot_peaks=True
                        ):
    
    
    peak_idxs, peaks_dict = find_peaks(stim_ch, height=[300, 3000], distance=100)
    pk_heights = peaks_dict["peak_heights"]
    events = []
    events_dict = { 
                    "PO70_pre": 11,
                    "PO75_pre": 12,
                    "PO80_pre": 13,
                    "PO85_pre": 14,
                    "PO90_pre": 15,               
                    "PO_bbn": 21,
                    "GO_bbn": 22,
                    "GP_bbn": 23,
                    "PO_3": 31,
                    "GO_3": 32,
                    "GP_3": 33,
                    "PO_8": 41,
                    "GO_8": 42,
                    "GP_8": 43,
                    "PO70_post": 51,
                    "PO75_post": 52,
                    "PO80_post": 53,
                    "PO85_post": 54,
                    "PO90_post": 55,
                    }
    match stim_key:
        case "gappre" | "gappost":
            height_limits = {
                            "PO70": [300, 650],
                            "PO75": [650, 1100],
                            "PO80": [1100, 1500],
                            "PO85": [1500, 1900],
                            "PO90": [1900, 3000],
                            }
        case "gapbbn" | "gap3" | "gap8":
            height_limits = {
                            "PO": [1700, 3000],
                            "GO": [300, 900],
                            "GP": [1200, 1700],
                            }   
    
    events = _detect_peaks(pk_heights, peak_idxs, stim_ch, times, height_limits, events_dict, stim_key, plot_peaks=plot_peaks)
    return events, events_dict

def _detect_peaks(pk_heights,
                peak_idxs,
                stim_ch,
                times,
                height_limits,
                events_dict,
                stim_key,
                plot_peaks=True
                ):
    events = []
    if plot_peaks:
        fig, ax = plt.subplots(1, 1, figsize=(12, 6))
        ax.plot(times, stim_ch)
        ax.set_title(stim_key)
        
    for key, (lower, upper) in height_limits.items():
        pk_idxs = np.where((pk_heights >= lower) & (pk_heights < upper))[0]
        sub_events = np.zeros(shape=(len(pk_idxs), 3), dtype=int)
        sub_events[:, 0] = times[peak_idxs[pk_idxs]] * 250
        sub_events[:, 1] = 0
        sub_events[:, 2] = events_dict[f"{key[:4]}_{stim_key[3:]}"]
        events.append(sub_events) 

        if plot_peaks:
            ax.scatter(times[peak_idxs[pk_idxs]], pk_heights[pk_idxs])

        if not len(pk_idxs) in [25, 100]:
            warnings.warn(f"\033[91mThe number of detected triggeres for {key} in {stim_key} is " \
                            "not as expected (25 or 100), its {len(pk_idxs)}, try adjusting the threshold!", UserWarning) 
            
    return np.concatenate(np.array(events), axis=0) 

In [66]:
event_ids

{'Stimulus/S  1': 2,
 'Stimulus/S  2': 3,
 'Stimulus/S  3': 4,
 'Stimulus/S  4': 5,
 'Stimulus/S  5': 6}

In [ ]:
stims, stim_ids = events, event_ids
stim_ids = {
            'gappre': 1,
            'gapbbn': 2,
            'gap3': 3,
            'gap8': 4,
            'gappost': 5,
            }
events = []
for stim_idx, stim in enumerate(stims):
    
    ## split recording into blocks
    start = stim[0]
    if stim_idx == len(stims) - 1:
        stop = None
    else: 
        stop = stims[stim_idx + 1][0]

    stim_ch, times = raw.get_data(picks="Audio",
                                start=start,
                                stop=stop,
                                return_times=True)
                                
    stim_ch = np.squeeze(stim_ch)
    stim_id = stim[2]
    stim_key = [key for key, value in stim_ids.items() if value == stim_id - 1][0] # maybe later better write it
    evs, event_ids = _detect_gpias_events(stim_ch, times, stim_key)

['gappre']
['gap3']
['gap8']
['gapbbn']
['gappost']


5

In [60]:
stim_id

6

In [22]:
stims

array([[  1228,      0,      2],
       [108385,      0,      4],
       [365875,      0,      5],
       [623901,      0,      3],
       [882334,      0,      6]])

Eyetracking

In [ ]:
import mne
raw = mne.io.read_raw_brainvision("/Users/payamsadeghishabestari/antinomics_clean_codes/subjects/zdfy/EEG/xxxxx/zdfy_xxxxx.vhdr")
mne.read_events(raw)

In [ ]:
raw.annotations

In [ ]:
from mne.io import read_raw_brainvision, read_raw_eyelink
from mne import events_from_annotations
from mne.preprocessing import realign_raw


## load
fname_eeg = "/Users/payamsadeghishabestari/Downloads/alertness_01_H036.vhdr"
fname_et = "/Users/payamsadeghishabestari/Downloads/local_al1H036.asc"

raw_eeg = read_raw_brainvision(fname_eeg, preload=True)
raw_eye = read_raw_eyelink(fname=fname_et, create_annotations=True) 
events_eeg, event_ids_eeg = events_from_annotations(raw_eeg)
events_eye, event_ids_eye = events_from_annotations(raw_eye)

## for now
stim_id_eeg_1, stim_id_eeg_2 = 8, 9
stim_id_eye_1, stim_id_eye_2 = 4, 5

s_raw = events_eeg[(events_eeg[:,2] == stim_id_eeg_1) | (events_eeg[:,2] == stim_id_eeg_2)][:, 0]
s_other = events_eye[(events_eye[:,2] == stim_id_eye_1) | (events_eye[:,2] == stim_id_eye_2)][:, 0]

## realigning
realign_raw(raw=raw_eeg,
            other=raw_eye,
            t_raw=s_raw / raw_eeg.info["sfreq"] - raw_eeg.first_time,
            t_other=s_other / raw_eye.info["sfreq"] - raw_eye.first_time,
            verbose=None)

raw_eye.add_channels([raw_eeg], force_update_info=True)
events, event_dict = events_from_annotations(raw_eye)
del raw_eeg  

In [ ]:
fname_eeg = "/Users/payamsadeghishabestari/Downloads/alertness_01_H036.vhdr"
fname_et = "/Users/payamsadeghishabestari/Downloads/local_al1H036.asc"

raw_eeg = read_raw_brainvision(fname_eeg, preload=True)
raw_eye = read_raw_eyelink(fname=fname_et, create_annotations=True) 

In [ ]:
raw_eye.plot()

In [ ]:
raw_eye.plot()